In [1]:
import os

os.environ["GSUTIL_PATH"] = "/nas/ucb/abhay/google-cloud-sdk/bin/gsutil"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

from src.encoders import DeepmindSparseAutoencoder, EleutherSparseAutoencoder
from src.utils import *

#encoder = DeepmindSparseAutoencoder.load_gemma2_sae(0, 11 )

encoder = EleutherSparseAutoencoder.load_llama3_sae(24, v2=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
output = encoder.featurize_text(["Hello world", "My name is abhay"])
feature_indices, feature_activations = output['model.layers.24']

print(feature_indices.shape)
print(feature_activations.shape)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


torch.Size([2, 6, 192])
torch.Size([2, 6, 192])


In [3]:
from src.database import FeatureDatabase
from datasets import load_dataset


database = FeatureDatabase(encoder)

if not os.path.exists("cached_sae_acts/"):
    # Load dataset
    dataset = load_dataset(
        "togethercomputer/RedPajama-Data-1T-Sample",
        split="train",
        # TODO: Maybe set this to False by default? But RPJ requires it.
        trust_remote_code=True,
    )
    # Cache SAE activations over dataset
    database.process_dataset(
        dataset=dataset,
        n_examples=100_000,
        folder_name="cached_sae_acts/",
        seed=42,
        example_seq_len=128,
        batch_size=64,
        text_column="text"
    )
else:
    database.load_from_disk("cached_sae_acts/")

In [4]:
ex = database.load_example(10)
print(ex)
print(len(ex.str_tokens))


<|begin_of_text|> all should burn for each—intensely, daily, and through word and work.”
“The last track, “Simulacrum“, closes this album. This song in so many ways cinches the thread tightly, both sonically and thematically, finishing this tapestry of ours. Contextually, it stands within the dualism and nihilism ever-present, the dance of life, and, the only enduring element of ourselves…our bone. That which will be stripped, bare, certain, and yet, hard enough persist. After all, this life is a dream…savor the seconds, the pain, and the glories
128


/nas/ucb/abhay/sae_experiments/src/database.py:117: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  result[hook_name] = (torch.from_numpy(latent_indices), torch.from_numpy(latent_acts))


In [5]:
from src.helper_classes import Feature, Example

common_features = database.get_common_features(encoder.hook_name, k=1000)


  0%|          | 0/98 [00:00<?, ?it/s]

In [10]:
from src.visualization import *
from IPython.display import HTML, display

    
def get_example_display(feature):
    max_activation_examples = feature.get_max_activating(20)
    display_dict = {"Top Activations": [ex.get_tokens_feature_lists(feature) for ex in max_activation_examples]}
    quantiles = feature.get_quantiles(8, 5)
    for i, (lower, upper) in enumerate(list(quantiles)[::-1]):
        examples = quantiles[(lower, upper)]
        display_dict[f"Interval {i} - ({lower:.2f}, {upper:.2f})"] = [ex.get_tokens_feature_lists(feature) for ex in examples]
    return generate_categorized_examples(display_dict)
    
light_mode_wrapper = """
<div style="background-color: white; color: black; padding: 20px;">
    {}
</div>
"""

my_html = get_example_display(common_features[1])
display(HTML(light_mode_wrapper.format(my_html)))

In [7]:
print(my_html)

<!DOCTYPE html><html><head>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            font-size: 14px;
            line-height: 1.3;
            margin: 0;
            padding: 10px;
        }
        .highlight {
            border-radius: 2px;
            padding: 0 1px;
        }
        .category {
            margin-bottom: 20px;
        }
        .category-label {
            font-weight: bold;
            font-size: 16px;
            color: #333;
            margin-bottom: 10px;
            border-bottom: 2px solid #333;
            padding-bottom: 5px;
        }
        .example {
            margin-bottom: 10px;
            border-bottom: 1px solid #eee;
            padding-bottom: 5px;
        }
        .example-label {
            font-weight: bold;
            font-size: 12px;
            color: #666;
            margin-bottom: 2px;
        }
        .text-content {
            word-wrap: break-word;
            overflo

In [8]:
list(common_features[1].get_quantiles(5,5))[::-1]

[(1.046875, 1.046875),
 (0.64599609375, 1.046875),
 (0.459716796875, 0.64599609375),
 (0.352294921875, 0.459716796875),
 (0.129150390625, 0.352294921875)]

In [9]:
common_features[1].get_logits()

([('self', 0.3664427399635315),
  (' self', 0.3624573051929474),
  ('-self', 0.326514333486557),
  (' selves', 0.3182579576969147),
  (' Self', 0.3110252320766449),
  ('Self', 0.30231282114982605),
  ('SELF', 0.2992872893810272),
  (' SELF', 0.2647799849510193)],
 [('ılıç', -0.06395630538463593),
  ('CastException', -0.06387549638748169),
  ('vise', -0.06376615911722183),
  ('olik', -0.0613018237054348),
  ('�名', -0.059393756091594696),
  ('erosis', -0.05870869383215904),
  ('ylko', -0.058203425258398056),
  ('ovice', -0.057864997535943985)])